In [ ]:
%pylab notebook
#Remove hashtags from the 2 lines below if you're not using magic
#import numpy as np
#import matplotlib.pyplot as plt
from astropy.table import Table
from uncertainties import ufloat
from uncertainties.umath import *
import read_mist_models #must have `read_mist_models.py` downloaded and in the same location as this notebook
import pandas as pd
import matplotlib.colors as mcolors

In [ ]:
#Reading in table with analysed stars
analysed_stars = Table.read('analysed_stars.csv', format = 'csv') #must have `analysed_stars.csv` downloaded and in the same location as this notebook

In [ ]:
#Creating ufloat variables to incorporate uncertainties in further calculations
para = []
for row in analysed_stars:
    p = ufloat(row['parallax'],row['error_parallax'])
    para.append(p)

extinct_mag = []
for row in analysed_stars:
    em = ufloat(row['A_G'],row['error_A_G'])
    extinct_mag.append(em)

excess_col = []
for row in analysed_stars:
    ec = ufloat(row['E(BP-RP)'],row['error_E(BP-RP)'])
    excess_col.append(ec)
    
sf_G = []
for row in analysed_stars:
    sG = ufloat(row['scale_factor_G'],row['error_scale_factor_G'])
    sf_G.append(sG)
    
sf_BP = []
for row in analysed_stars:
    sB = ufloat(row['scale_factor_BP'],row['error_scale_factor_BP'])
    sf_BP.append(sB)
    
sf_RP = []
for row in analysed_stars:
    sR = ufloat(row['scale_factor_RP'],row['error_scale_factor_RP'])
    sf_RP.append(sR)

fr_G = []
for row in analysed_stars:
    fG = ufloat(row['flux_ratio_G'],row['error_flux_ratio_G'])
    fr_G.append(fG)
    
fr_BP = []
for row in analysed_stars:
    fB = ufloat(row['flux_ratio_BP'],row['error_flux_ratio_BP'])
    fr_BP.append(fB)
    
fr_RP = []
for row in analysed_stars:
    fR = ufloat(row['flux_ratio_RP'],row['error_flux_ratio_RP'])
    fr_RP.append(fR)

In [ ]:
#Converting scale factor values into flux
scale_factor_Ginflux = []
for sG in sf_G:
    scale_factor_Ginflux.append(10**(sG/-2.5))

scale_factor_BPinflux = []
for sB in sf_BP:
    scale_factor_BPinflux.append(10**(sB/-2.5))

scale_factor_RPinflux = []
for sR in sf_RP:
    scale_factor_RPinflux.append(10**(sR/-2.5))

In [ ]:
#Calculating primary and secondary contributions in each colour band
primary_cont_G = []
for fG in fr_G:
    primary_cont_G.append(1/(fG+1))

secondary_cont_G = []
for fG in fr_G:
    secondary_cont_G.append(fG/(fG+1))

primary_cont_BP = []
for fB in fr_BP:
    primary_cont_BP.append(1/(fB+1))

secondary_cont_BP = []
for fB in fr_BP:
    secondary_cont_BP.append(fB/(fB+1))

primary_cont_RP = []
for fR in fr_RP:
    primary_cont_RP.append(1/(fR+1))

secondary_cont_RP = []
for fR in fr_RP:
    secondary_cont_RP.append(fR/(fR+1))

In [ ]:
#Calculating apparent magnitude of each star in each colour band
mag_hot_G = []
for sfact, pcont in zip(scale_factor_Ginflux, primary_cont_G):
    mag_hot_G.append(-2.5*log10(sfact*pcont))

mag_hot_BP = []
for sfact, pcont in zip(scale_factor_BPinflux, primary_cont_BP):
    mag_hot_BP.append(-2.5*log10(sfact*pcont))

mag_hot_RP = []
for sfact, pcont in zip(scale_factor_RPinflux, primary_cont_RP):
    mag_hot_RP.append(-2.5*log10(sfact*pcont))

mag_faint_G = []
for sfact, scont in zip(scale_factor_Ginflux, secondary_cont_G):
    mag_faint_G.append(-2.5*log10(sfact*scont))

mag_faint_BP = []
for sfact, scont in zip(scale_factor_BPinflux, secondary_cont_BP):
    mag_faint_BP.append(-2.5*log10(sfact*scont))

mag_faint_RP = []
for sfact, scont in zip(scale_factor_RPinflux, secondary_cont_RP):
    mag_faint_RP.append(-2.5*log10(sfact*scont))

In [ ]:
#Converting magnitude in G band to absolute magnitude for plotting on y-axis
d_parsecs = []
for p in para:
    d_parsecs.append(1 / (p*10**-3))  #Converting from mas to parsecs
        
abs_mag_hot_G = []
for mG, d, ext in zip(mag_hot_G, d_parsecs, extinct_mag):
    abs_mag_hot_G.append(mG - 5*(log10(0.1*d)) - ext)

abs_mag_faint_G = []
for mG, d, ext in zip(mag_faint_G, d_parsecs, extinct_mag):
    abs_mag_faint_G.append(mG - 5*(log10(0.1*d)) - ext)

In [ ]:
#Calculating colour index for plotting on x-axis
colour_index_hot = []
for mBP, mRP, exc in zip(mag_hot_BP, mag_hot_RP, excess_col):
    colour_index_hot.append(mBP - mRP - exc)

colour_index_faint = []
for mBP, mRP, exc in zip(mag_faint_BP, mag_faint_RP, excess_col):
    colour_index_faint.append(mBP - mRP - exc)

In [ ]:
#Reading in isochrones
isocmd = read_mist_models.ISOCMD('MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_UBVRIplus.iso.cmd')

#Indices of four isochrone ages
AgeOne = isocmd.age_index(np.log10(0.5*10**9)) # 0.5Gyr
AgeTwo = isocmd.age_index(np.log10(1*10**9)) # 1Gyr
AgeThree = isocmd.age_index(np.log10(5*10**9)) # 5Gyr
AgeFour = isocmd.age_index(np.log10(10*10**9)) # 10Gyr

#Four isochrones
BP_one = isocmd.isocmds[AgeOne]['Gaia_BP_EDR3']
RP_one = isocmd.isocmds[AgeOne]['Gaia_RP_EDR3']
G_one = isocmd.isocmds[AgeOne]['Gaia_G_EDR3']

BP_two = isocmd.isocmds[AgeTwo]['Gaia_BP_EDR3']
RP_two = isocmd.isocmds[AgeTwo]['Gaia_RP_EDR3']
G_two = isocmd.isocmds[AgeTwo]['Gaia_G_EDR3']

BP_three = isocmd.isocmds[AgeThree]['Gaia_BP_EDR3']
RP_three = isocmd.isocmds[AgeThree]['Gaia_RP_EDR3']
G_three = isocmd.isocmds[AgeThree]['Gaia_G_EDR3']

BP_four = isocmd.isocmds[AgeFour]['Gaia_BP_EDR3']
RP_four = isocmd.isocmds[AgeFour]['Gaia_RP_EDR3']
G_four = isocmd.isocmds[AgeFour]['Gaia_G_EDR3']

We plotted our finalized HRD over the Gaia HRD referenced from [here](https://sci.esa.int/web/gaia/-/60198-gaia-hertzsprung-russell-diagram). This was achieved by inserting the following into the ADQL advanced query on Gaia Archive to obtain a csv file containing data of a set of stars which have been filtered based on what this [paper](https://www.aanda.org/articles/aa/full_html/2018/08/aa32843-18/aa32843-18.html#S3) did (Section 2.1):

SELECT phot_g_mean_mag+5*log10(parallax)-10 AS mg, bp_rp FROM gaiadr2.gaia_source
	WHERE parallax_over_error > 10
	AND phot_g_mean_flux_over_error>50
	AND phot_rp_mean_flux_over_error>20
	AND phot_bp_mean_flux_over_error>20
	AND phot_bp_rp_excess_factor < 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
	AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
	AND visibility_periods_used>8
	AND astrometric_chi2_al/(astrometric_n_good_obs_al-5)<1.44*greatest(1,exp(-0.4*(phot_g_mean_mag-19.5)))
	AND e_bp_min_rp_val < 0.300
	AND random_index BETWEEN 0 AND 10000000

In [ ]:
#Reading in the GAIA HRD csv file
data = pd.read_csv('GaiaHRD2.csv') #must have 'GaiaHRD2.csv' downloaded and in the same location as this notebook

In [ ]:
#Plotting the CMD
fig, ax = plt.subplots(figsize=(7.5,7))
symbol = ['s', 'o', 'x', 'p', '*', '^', '.', 'd','P', 'h']

#Plotting the hotter stars
for i, star in enumerate(analysed_stars):
    color_index = colour_index_hot[i].n
    abs_mag = abs_mag_hot_G[i].n
    xerr = colour_index_hot[i].s
    yerr = abs_mag_hot_G[i].s
    sym = symbol[i % len(symbol)]
    ax.errorbar(color_index, abs_mag, xerr=xerr, yerr=yerr, ms=7, fmt=sym, color='red')
    
#Plotting the fainter stars
for j, star in enumerate(analysed_stars):
    color_index = colour_index_faint[j].n
    abs_mag = abs_mag_faint_G[j].n
    xerr = colour_index_faint[j].s
    yerr = abs_mag_faint_G[j].s
    sym = symbol[j % len(symbol)]
    ax.errorbar(color_index, abs_mag, xerr=xerr, yerr=yerr, ms=7, fmt=sym, color='blue')

#Plotting the isochrones
ax.plot(BP_one - RP_one, G_one, c='black', label='Age = 0.5 Gyr', linestyle='dotted', alpha=0.4)

ax.plot(BP_two - RP_two, G_two, c='black', label='Age = 1 Gyr', linestyle='dashed', alpha=0.4)

ax.plot(BP_three - RP_three, G_three, c='black', label='Age = 5 Gyr', linestyle='dashdot', alpha=0.4)

ax.plot(BP_four - RP_four, G_four, c='black', label='Age = 10 Gyr', linestyle='solid', alpha=0.4)
    
#Plotting the GAIA CMD
plt.scatter(data['bp_rp'], data['mg'], s=2, c='darkgray', alpha=0.3)

#Other plot specifications
ax.invert_yaxis()
ax.set_xlim(-0.05, 1.1)
ax.set_ylim(6, 1.4)
ax.set_xlabel('Colour Index, BP-RP (mag)')
ax.set_ylabel('Absolute Magnitude, G (mag)')

plt.show()

In [ ]:
plt.savefig('CMD.pdf')